In [1]:
from safe_control_gym.envs.manipulators.manipulator import BaseManipulator
import numpy as np 
import random 
from time import sleep
from safe_control_gym.envs.benchmark_env import Cost, Task
from gym import spaces


pybullet build time: Mar 29 2022 22:56:14


In [2]:
urdf_path = "../safe_control_gym/envs/manipulators/assets/franka_panda/panda.urdf"
control_mode = "torque"
target_space = "joint"

controlled_joint_indices = [6]
observed_link_indices = [6]
observed_link_state_keys = ["orientation"]
goal = [{
    "orientation": [None for i in range(13)]
}]

orientation_goal = [0.603, 0.3687, -0.3697, 0.6026] 
goal[0]["orientation"][observed_link_indices[0]] = orientation_goal
goal_type = "point"

env = BaseManipulator(
    urdf_path,
    control_mode,
    target_space,
    controlled_joint_indices = controlled_joint_indices,
    observed_link_indices = observed_link_indices, 
    observed_link_state_keys = observed_link_state_keys,
    goal = goal,
    goal_type = goal_type,
    cost = Cost.RL_REWARD
)

argv[0]=
Version = 4.1 Metal - 76.3
Vendor = Apple
Renderer = Apple M1 Pro
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [3]:
env.link_states

{'position': [[], [], [], [], [], [], [], [], [], [], [], [], []],
 'orientation': [[], [], [], [], [], [], [], [], [], [], [], [], []],
 'linear_vel': [[], [], [], [], [], [], [], [], [], [], [], [], []],
 'angular_vel': [[], [], [], [], [], [], [], [], [], [], [], [], []]}

In [3]:
# action_space = [ -200, -300, -400, 200, 300, 400,]
action_space = [0]

for i in range(50):
#     action_index = env.n_joints-2
    action_index = 6
    action_list = np.zeros(env.n_joints)
    action_list[action_index] = random.choice(action_space)    
    obs,reward, done,info = env.step(action_list)
    if i%5== 0 :
          print(f"obs: {obs}") # link position
          print(f"reward: {reward}")
        #   print(f"angular velocity: {obs}") # velocity
                
    sleep(0.05)

obs: (0.7071067690849304, 0.0, 0.0, 0.7071067690849304)
reward: -0.9470135381698608
obs: (0.7071067690849304, 0.0, 0.0, 0.7071067690849304)
reward: -0.9470135381698608
obs: (0.7071067690849304, 0.0, 0.0, 0.7071067690849304)
reward: -0.9470135381698608
obs: (0.7071067690849304, 0.0, 0.0, 0.7071067690849304)
reward: -0.9470135381698608
obs: (0.7071067690849304, 0.0, 0.0, 0.7071067690849304)
reward: -0.9470135381698608
obs: (0.7071067690849304, 0.0, 0.0, 0.7071067690849304)
reward: -0.9470135381698608
obs: (0.7071067690849304, 0.0, 0.0, 0.7071067690849304)
reward: -0.9470135381698608
obs: (0.7071067690849304, 0.0, 0.0, 0.7071067690849304)
reward: -0.9470135381698608
obs: (0.7071067690849304, 0.0, 0.0, 0.7071067690849304)
reward: -0.9470135381698608
obs: (0.7071067690849304, 0.0, 0.0, 0.7071067690849304)
reward: -0.9470135381698608


In [ ]:
env.n_joints

13

In [ ]:
link_states =  env._pb_client.getLinkState(env.robot,
                                            linkIndex=6,
                                            computeLinkVelocity=True)


In [ ]:
link_states

((0.039833163452802724, -0.005246717093960421, 1.0411480839438747),
 (0.6968757615251299,
  -0.11944479553082608,
  0.025003790002564637,
  0.7067332766361558),
 (0.04, 0.0, 0.0),
 (0.0, 0.0, 0.0, 1.0),
 (0.0010245442390441895, -1.3434328138828278e-06, 1.0330008268356323),
 (0.6968757510185242,
  -0.11944480240345001,
  0.025003790855407715,
  0.7067332863807678),
 (0.16140478353054072, -0.021992152304630256, -0.7820516002624263),
 (2.675834999501902, 19.791254360890452, -0.004319621550829485))

In [25]:
# only extract world positions
link_states_dict = {
    "position": link_states[4],
    "orientation": link_states[5],
    "linear_vel": link_states[6],
    "angular_vel": link_states[7]
}

In [26]:
link_states_dict

{'position': (0.0010245442390441895,
  -1.3434328138828278e-06,
  1.0330008268356323),
 'orientation': (0.6968757510185242,
  -0.11944480240345001,
  0.025003790855407715,
  0.7067332863807678),
 'linear_vel': (0.16140478353054072,
  -0.021992152304630256,
  -0.7820516002624263),
 'angular_vel': (2.675834999501902, 19.791254360890452, -0.004319621550829485)}

In [4]:
env.link_states["position"][1] = [4,3,2]

In [ ]:
# point 
# 1.0330

# orientation:
# (0.6034725904464722, 0.3687118589878082, -0.3697621822357178, 0.6026179194450378)

In [9]:
from gym import spaces 
# env.observation_space = spaces.Box(-np.inf, np.inf, (14,), dtype=np.float32)

# orientation use quaternion so 4 dim 

# action_space = [spaces.Discrete(1) for i in range(env.n_joints-1)]
# action_space.append(spaces.Box(-400, 400, shape=(1,), dtype=np.float32))

# env.action_space = spaces.Tuple(
#     action_space
# )

# dim dict
dim_dict = {
    "orientation" : 4 
}
env.action_space = spaces.Box(-300, -200, (env.n_joints,), dtype=np.float32)

# TODO expand to multiple state keys 
env.observation_space = spaces.Box(-np.inf, np.inf, (dim_dict[observed_link_state_keys[0]], ), dtype=np.float32)

# env.observation_space = spaces.Dict({
#       'position': spaces.Box(-np.inf, np.inf, (env.n_joints, 3), dtype=np.float32),
#       'orientation': spaces.Box(-np.inf, np.inf, (env.n_joints, 4), dtype=np.float32),
#       'linear_vel': spaces.Box(-np.inf, np.inf, (env.n_joints, 3), dtype=np.float32),
#       'angular_vel': spaces.Box(-np.inf, np.inf, (env.n_joints, 3), dtype=np.float32),
#     })


In [10]:
from stable_baselines3 import A2C
import gym 
# model = PPO('MultiInputPolicy', env, verbose=1)
# model.learn(total_timesteps=10)
# env = gym.make('CartPole-v1')

model = A2C('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=1000)




Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| time/                 |          |
|    fps                | 55       |
|    iterations         | 100      |
|    time_elapsed       | 9        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -18.6    |
|    explained_variance | 2.98e-07 |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -213     |
|    std                | 1.01     |
|    value_loss         | 148      |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 55       |
|    iterations         | 200      |
|    time_elapsed       | 18       |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -18.6    |
|    explained_variance | 0        |
|    learning_rate

In [11]:
obs = env.reset()
for i in range(10):
    action, _state = model.predict(obs, deterministic=True)
    print(f"action: {action}")
    print(f"goal: {goal[0]['orientation'][6]}")
    obs, reward, done, info = env.step(action)
    print(f"state: {obs}")
    print(f"reward: {reward}")
    sleep(0.05)

action: [-200. -200. -200. -200. -200. -200. -200. -200. -200. -200. -200. -200.
 -200.]
goal: [0.603, 0.3687, -0.3697, 0.6026]
state: (-0.27052879333496094, -0.6532025933265686, 0.6544196605682373, -0.2680959105491638)
reward: -3.7902469577789306
action: [-200. -200. -200. -200. -200. -200. -200. -200. -200. -200. -200. -200.
 -200.]
goal: [0.603, 0.3687, -0.3697, 0.6026]
state: (-0.27053022384643555, -0.6532018780708313, 0.6544206142425537, -0.26809409260749817)
reward: -3.7902468087673187
action: [-200. -200. -200. -200. -200. -200. -200. -200. -200. -200. -200. -200.
 -200.]
goal: [0.603, 0.3687, -0.3697, 0.6026]
state: (-0.27053162455558777, -0.6532010436058044, 0.6544215679168701, -0.26809221506118774)
reward: -3.79024645113945
action: [-200. -200. -200. -200. -200. -200. -200. -200. -200. -200. -200. -200.
 -200.]
goal: [0.603, 0.3687, -0.3697, 0.6026]
state: (-0.2705330550670624, -0.6532003283500671, 0.6544225215911865, -0.2680903971195221)
reward: -3.790246302127838
action: [-

In [5]:
# missing observation_space and action_space 

In [9]:
from gym import spaces 
_action_space = spaces.Box(0, 2, (2,2), dtype=np.float32)


In [10]:
_action_space

Box([[0. 0.]
 [0. 0.]], [[2. 2.]
 [2. 2.]], (2, 2), float32)